**Evaluation with GPT2**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt update
!sudo apt install openjdk-17-jdk -y
!sudo update-alternatives --install /usr/bin/java java /usr/lib/jvm/java-17-openjdk-amd64/bin/java 1
!sudo update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java

# Verify the version
!java -version


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,605 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,696 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jam

In [3]:
!pip install language-tool-python spacy transformers torch nltk pandas openpyxl tqdm
import spacy
import language_tool_python
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import nltk
import pandas as pd
from tqdm import tqdm

# Download required data
nltk.download('punkt')
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    !python -m spacy download en_core_web_sm
    nlp = spacy.load('en_core_web_sm')

class GrammarEvaluator:
    def __init__(self):
        """Initialize the grammar evaluator with necessary tools"""
        try:
            self.language_tool = language_tool_python.LanguageTool('en-US')
            self.nlp = spacy.load('en_core_web_sm')
            self.gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
            self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            self.gpt2_model.to(self.device)
            print("Successfully initialized all components")
        except Exception as e:
            print(f"Error initializing components: {str(e)}")
            raise

    def calculate_error_density(self, text):
        try:
            matches = self.language_tool.check(text)
            word_count = len(text.split())
            error_count = len(matches)
            error_details = []
            for match in matches:
                error_details.append({
                    'message': match.message,
                    'replacements': match.replacements[:3],
                    'context': match.context,
                    'category': match.category
                })
            return {
                'error_count': error_count,
                'word_count': word_count,
                'error_density': (error_count / word_count * 100) if word_count > 0 else 0,
                'error_details': error_details
            }
        except Exception as e:
            print(f"Error calculating error density: {str(e)}")
            return None

    def calculate_complexity_metrics(self, text):
        try:
            doc = self.nlp(text)
            words = [token.text for token in doc if not token.is_punct]
            sentences = list(doc.sents)

            def get_depth(token):
                return 1 + max([get_depth(child) for child in token.children], default=0)

            metrics = {
                'avg_word_length': np.mean([len(word) for word in words]) if words else 0,
                'avg_sentence_length': np.mean([len([token for token in sent if not token.is_punct])
                                             for sent in sentences]) if sentences else 0,
                'unique_words_ratio': len(set(words)) / len(words) if words else 0,
                'pos_distribution': dict(nltk.FreqDist([token.pos_ for token in doc])),
                'avg_syntax_depth': np.mean([get_depth(sent.root) for sent in sentences]) if sentences else 0
            }
            return metrics
        except Exception as e:
            print(f"Error calculating complexity metrics: {str(e)}")
            return None

    def calculate_readability(self, text):
        try:
            def count_syllables(word):
                word = word.lower()
                count = 0
                vowels = 'aeiouy'
                if word[0] in vowels:
                    count += 1
                for index in range(1, len(word)):
                    if word[index] in vowels and word[index-1] not in vowels:
                        count += 1
                if word.endswith('e'):
                    count -= 1
                if count == 0:
                    count += 1
                return count

            doc = self.nlp(text)
            words = [token.text for token in doc if not token.is_punct]
            sentences = list(doc.sents)

            word_count = len(words)
            sentence_count = len(sentences)
            syllable_count = sum(count_syllables(word) for word in words)
            complex_words = len([word for word in words if count_syllables(word) >= 3])

            if word_count > 0 and sentence_count > 0:
                flesch = 206.835 - 1.015 * (word_count/sentence_count) - 84.6 * (syllable_count/word_count)
                gunning_fog = 0.4 * ((word_count/sentence_count) + 100 * (complex_words/word_count))
            else:
                flesch = 0
                gunning_fog = 0

            return {
                'flesch_reading_ease': flesch,
                'gunning_fog_index': gunning_fog,
                'avg_syllables_per_word': syllable_count/word_count if word_count > 0 else 0,
                'complex_word_ratio': complex_words/word_count if word_count > 0 else 0
            }
        except Exception as e:
            print(f"Error calculating readability: {str(e)}")
            return None

    def calculate_perplexity(self, text):
        try:
            encodings = self.gpt2_tokenizer(text, return_tensors='pt')
            input_ids = encodings.input_ids.to(self.device)
            with torch.no_grad():
                outputs = self.gpt2_model(input_ids, labels=input_ids)
                loss = outputs.loss
            return torch.exp(loss).item()
        except Exception as e:
            print(f"Error calculating perplexity: {str(e)}")
            return None

    def evaluate(self, text):
        try:
            error_metrics = self.calculate_error_density(text)
            complexity_metrics = self.calculate_complexity_metrics(text)
            readability_metrics = self.calculate_readability(text)
            perplexity = self.calculate_perplexity(text)

            error_score = max(0, 1 - error_metrics['error_density']/10)
            complexity_score = min(1, complexity_metrics['avg_syntax_depth']/5)
            readability_score = min(1, max(0, readability_metrics['flesch_reading_ease']/100))
            perplexity_score = max(0, 1 - (perplexity-20)/100)

            final_score = (error_score * 0.4 +
                         complexity_score * 0.2 +
                         readability_score * 0.2 +
                         perplexity_score * 0.2) * 100

            return {
                'final_score': final_score,
                'subscores': {
                    'grammar': error_score * 100,
                    'complexity': complexity_score * 100,
                    'readability': readability_score * 100,
                    'fluency': perplexity_score * 100
                },
                'detailed_metrics': {
                    'error_metrics': error_metrics,
                    'complexity_metrics': complexity_metrics,
                    'readability_metrics': readability_metrics,
                    'perplexity': perplexity
                }
            }
        except Exception as e:
            print(f"Error in evaluation: {str(e)}")
            return None

# Process Excel file
input_file = '/content/drive/MyDrive/MC_Data/generated_items/GPT.xlsx'
output_file = '/content/drive/MyDrive/MC_Data/generated_items/evaluation/All_str_GPT.xlsx'

# Initialize evaluator
evaluator = GrammarEvaluator()

try:
    # Read Excel file
    print(f"Reading file: {input_file}")
    df = pd.read_excel(input_file)

    # Concatenate specified columns
    print("Processing sentences...")
    df['combined_text'] = df[['question']].fillna('').astype(str).apply(' '.join, axis=1)
    df['combined_text'] = df['combined_text'].astype(str).str.capitalize()
    df['question'] = df['question'].astype(str).str.capitalize()

    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        text = row['combined_text'].strip()
        if text:
            evaluation = evaluator.evaluate(text)
            if evaluation:
                results.append({
                    'Row': idx + 2,  # Excel row number (1-based + header)
                    'Prompting_strategy':row.get('strategy', ''),
                    'Question_Type':row.get('question_type', ''),
                    'Question':row.get('question',''),
                    'Correct_Answer': row.get('correct_answer', ''),
                    'Choice_1': row.get('choice_a', ''),
                    'Choice_2': row.get('choice_b', ''),
                    'Choice_3': row.get('choice_c', ''),
                    'Word_Difficulty': row.get('word_difficulty', ''),
                    'Task_Difficulty': row.get('task_difficulty', ''),
                    'Text': text,
                    'Overall_Score': evaluation['final_score'],
                    'Grammar_Score': evaluation['subscores']['grammar'],
                    'Complexity_Score': evaluation['subscores']['complexity'],
                    'Readability_Score': evaluation['subscores']['readability'],
                    'Fluency_Score': evaluation['subscores']['fluency'],
                    'Error_Count': evaluation['detailed_metrics']['error_metrics']['error_count'],
                    'Errors': '; '.join([error['message'] for error in
                                       evaluation['detailed_metrics']['error_metrics']['error_details']])
                })

    # Create and save results DataFrame
    results_df = pd.DataFrame(results)
    results_df.to_excel(output_file, index=False)

    print("\nProcessing complete!")
    print(f"Total sentences processed: {len(results_df)}")
    print(f"Average score: {results_df['Overall_Score'].mean():.2f}")
    print(f"Total errors found: {results_df['Error_Count'].sum()}")

    # Display sample results
    print("\nSample results (first 5 rows):")
    print(results_df[['Row', 'Overall_Score', 'Error_Count', 'Errors']].head())

except Exception as e:
    print(f"Error processing file: {str(e)}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:language_tool_python.download_lt:Unzipping /tmp/tmpfizyklgv.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://internal1.languagetool.org/snapshots/LanguageTool-latest-snapshot.zip to /root/.cache/language_tool_python.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Successfully initialized all components
Reading file: /content/drive/MyDrive/MC_Data/generated_items/GPT.xlsx
Processing sentences...


100%|██████████| 3509/3509 [14:54<00:00,  3.92it/s]



Processing complete!
Total sentences processed: 3509
Average score: 79.16
Total errors found: 230

Sample results (first 5 rows):
   Row  Overall_Score  Error_Count Errors
0    2         70.769            0       
1    3         73.589            0       
2    4         73.589            0       
3    5         70.769            0       
4    6         73.589            0       
